# Visualize results, local splicing

## Load Libraries

In [1]:
library(tidyverse)
library(ggplot2)
library(DT)
library(leafcutter)
library(reshape2)
library(gridExtra)
library(intervals) # needed for pretty strand arrow placement
library(foreach)
library(grid)
library(gtable)
library(ggrepel)

-- Attaching packages ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.2     v dplyr   1.0.7
v tidyr   1.1.3     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Loading required package: Rcpp


Attaching package: 'reshape2'


The following object is masked from 'package:tidyr':

    smiths



Attaching package: 'gridExtra'


The following object is masked from 'package:dplyr':

    combine



Attaching package: 'intervals'


The following object is masked

## Summary of results

In [1]:
lname = load('../../_m/leafviz.RData')
lname

In [2]:
levels(meta$group) <- c("Female", "Male")
sample_table

In [4]:
cluster_summary

Results,n
<chr>,<int>
Number of differentially spliced clusters at FDR = 0.1,234
Fully annotated,84
Contain unannotated junctions,150


In [5]:
intron_summary

Results,n
<chr>,<int>
Number of fully annotated junctions,687
Number of junctions with cryptic 5' splice site,126
Number of junctions with cryptic 3' splice site,111
Number of junctions with two cryptic splice sites,14
Number of novel junctions that connect two annotated splice sites,89


In [6]:
clusters['gene'] <- gsub("</i>", "", gsub("<i>", "", clusters$gene))
head(clusters)

,clusterID,N,coord,gene,annotation,FDR
,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
232,clu_765_-,3,chrX:53217966-53220839,KDM5C,annotated,2.37e-38
230,clu_758_-,10,chrX:53176622-53193437,KDM5C,cryptic,1.10e-37
228,clu_66561_+,7,chrX:153771081-153771864,PLXNB3,cryptic,1.69e-28
223,clu_65232_+,3,chrX:47199106-47199480,UBA1,cryptic,5.23e-17
231,clu_763_-,3,chrX:53210576-53211497,KDM5C,cryptic,8.61e-11
176,clu_118132_+,10,chr6:125248383-125262834,TPD52L1,cryptic,1.18e-08


In [ ]:
write.table(clusters, file="cluster_ds_results_annotated.txt", sep="\t", quote=FALSE, row.names=FALSE)

## Generate plots

### Define functions

In [8]:
filter_intron_table <- function(introns, clu){
    d < - introns %>% filter(clusterID == clu) %>% 
        select(chr, start, end, verdict, deltapsi) %>%
        arrange(desc(abs(deltapsi))) %>%
        rename("ΔPSI" = deltapsi)
    row.names(d) <- letters[1:nrow(d)] # letters is just a:z
    return(d)
}


getGeneLength <- function(gene_name, exons_table){
    exons      <- exons_table[ exons_table$gene_name == gene_name, ]
    geneStart  <- min(exons$start)
    geneEnd    <- max(exons$end)
    geneLength <- geneEnd - geneStart
    if( geneLength >1e6){
        pixels <- 5000
    } else if ( geneLength > 5e5 & geneLength < 1e6){
        pixels <- 3000
    } else if ( geneLength > 1.5e5 & geneLength <= 5e5){
        pixels <- 2000
    } else {
        stopifnot(geneLength <= 1.5e5)
        pixels <- "auto"
    }
    return(pixels)
}


select_data <- function(sel, clusters, exons_table){
    gene <- clusters[ sel, ]$gene
    width <- getGeneLength(gene, exons_table)
    clusterID <- clusters[ sel, ]$clusterID
    coord <- clusters[ sel, ]$coord
    return(list(gene = gene, width = width, cluster = clusterID, coord = coord))
}

### Plot top 6 clusters

In [9]:
plot_cluster <- function(num, clusters, dir='./'){
    mydata = select_data(num, clusters, exons_table)
    while(mydata$gene == '.'){
        num = num+1
        mydata = select_data(num, clusters, exons_table)
    }
    gene_name = mydata$gene
    plotTitle <- paste0(gene_name, '_', mydata$cluster, '_top_',num,'.pdf')
    pdf(file=paste0(dir, plotTitle), width = 10, height = 5)
    print(make_cluster_plot(mydata$cluster,
                            main_title = plotTitle,
                            meta = meta,
                            cluster_ids = cluster_ids,
                            exons_table = exons_table,
                            counts = counts,
                            introns = introns))
    dev.off()

    if (is.numeric(mydata$width)) {
        new_width = mydata$width / 100
    } else {
        new_width = mydata$width
    }

    pdf(file=paste0(dir, gene_name, '_allClusters_top_',num,'.pdf'), 
        width=new_width, height=6)
    print(make_gene_plot(mydata$gene,
                         counts = counts,
                         introns = introns,
                         exons_table = exons_table,
                         cluster_list = clusters,
                         clusterID = mydata$clusterID,
                         introns_to_plot = introns_to_plot, debug=F))
    dev.off()
}


## Plot splicing

In [21]:
dir.create("top10")
for(num in 1:10){
    plot_cluster(num, clusters, "top10/")
}

TableGrob (2 x 1) "arrange": 2 grobs
  z     cells    name           grob
1 1 (1-1,1-1) arrange gtable[layout]
2 2 (2-2,1-1) arrange gtable[layout]


Warning message:
"'mode(width)' differs between new and previous
	 ==> NOT changing 'width'"


TableGrob (2 x 1) "arrange": 2 grobs
  z     cells    name           grob
1 1 (1-1,1-1) arrange gtable[layout]
2 2 (2-2,1-1) arrange gtable[layout]


Warning message:
"'mode(width)' differs between new and previous
	 ==> NOT changing 'width'"


TableGrob (2 x 1) "arrange": 2 grobs
  z     cells    name           grob
1 1 (1-1,1-1) arrange gtable[layout]
2 2 (2-2,1-1) arrange gtable[layout]
TableGrob (2 x 1) "arrange": 2 grobs
  z     cells    name           grob
1 1 (1-1,1-1) arrange gtable[layout]
2 2 (2-2,1-1) arrange gtable[layout]


Warning message:
"'mode(width)' differs between new and previous
	 ==> NOT changing 'width'"


TableGrob (2 x 1) "arrange": 2 grobs
  z     cells    name           grob
1 1 (1-1,1-1) arrange gtable[layout]
2 2 (2-2,1-1) arrange gtable[layout]
TableGrob (2 x 1) "arrange": 2 grobs
  z     cells    name           grob
1 1 (1-1,1-1) arrange gtable[layout]
2 2 (2-2,1-1) arrange gtable[layout]


In [22]:
dir.create("x_chromosome")
x_clu = clusters %>% filter(str_detect(coord, "chrX"), FDR < 0.05)

for(num in 1:dim(x_clu)[1]){
    plot_cluster(num, x_clu, "x_chromosome/")
}

In [10]:
dir.create("ank3")
ank3 = clusters %>% filter(gene == 'ANK3')
ank3

clusterID,N,coord,gene,annotation,FDR
<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
